In [1]:
import pandas as pd 
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
close_prices = pd.read_csv('../data/processed_data.csv', index_col='Date',
                           parse_dates=True)
print(close_prices)
tsla_prices = close_prices[['TSLA']].copy()
print(tsla_prices)

                  BND         SPY        TSLA
Date                                         
2015-07-01  60.816719  174.917130   17.943333
2015-07-02  60.967091  174.756958   18.667999
2015-07-06  61.177593  174.259644   18.648001
2015-07-07  61.237736  175.355484   17.858667
2015-07-08  61.410656  172.413498   16.997334
...               ...         ...         ...
2025-07-24  72.698860  634.419983  305.299988
2025-07-25  72.878265  637.099976  316.059998
2025-07-28  72.758659  636.940002  325.589996
2025-07-29  73.137398  635.260010  321.200012
2025-07-30  72.948036  634.460022  319.040009

[2535 rows x 3 columns]
                  TSLA
Date                  
2015-07-01   17.943333
2015-07-02   18.667999
2015-07-06   18.648001
2015-07-07   17.858667
2015-07-08   16.997334
...                ...
2025-07-24  305.299988
2025-07-25  316.059998
2025-07-28  325.589996
2025-07-29  321.200012
2025-07-30  319.040009

[2535 rows x 1 columns]


In [3]:
split_date = '2023-12-31'

train_data = tsla_prices.loc[tsla_prices.index <= split_date]
test_data = tsla_prices.loc[tsla_prices.index > split_date]

print(f"Training data points: {len(train_data)}")
print(f"Testing data points: {len(test_data)}")

Training data points: 2140
Testing data points: 395


In [4]:
warnings.filterwarnings('ignore')

In [5]:
from pmdarima import auto_arima

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
print("Finding optimal ARIMA parameters with auto_arima...")

arima_model = auto_arima(train_data['TSLA'], start_p=1, start_q=1, test='adf', max_p=5, max_q=5, m=1, d=None, seasonal=False, stat_p=0, D=0, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

Finding optimal ARIMA parameters with auto_arima...


NameError: name 'auto_arima' is not defined

In [ ]:
print("\nARIMA Model Summary:")
print(arima_model.summary())

In [ ]:
print("\nForecasting with ARIMA model...")

arima_forcasts = arima_model.predict(n_periods=len(test_data))
arima_forcasts = pd.Series(arima_forcasts, index=test_data.index)